In [8]:

from curl_cffi import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import os
import sys
from unidecode import unidecode
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import re

from dotenv import load_dotenv
load_dotenv()

local_path = os.getenv('RAW_PATH')
query_crawl = os.getenv('QUERY_CRAWL_PATH')

## Hàm lấy danh mục

### Danh mục parent - c1

In [9]:
# Lấy tất cả các danh mục cha - danh mục cấp 1
def crawl_all_categories_c1 ():

  url = 'https://www.fahasa.com/'

  response = requests.get(url, impersonate='chrome')

  soup = BeautifulSoup(response.content, 'html.parser')

  cate_books = soup.find_all('li', class_ = ['parent', 'dropdown' , 'parent dropdown aligned-left'])
  json_categories = []
  for cate in cate_books:
    title = cate.find('a')
    category = {}
    category['Tên danh mục c1'] = title.get_text()
    category['Liên kết'] = title.attrs['href']
    json_categories.append(category)

  df_categories = pd.json_normalize(json_categories)
  return df_categories

### Danh mục child - c2


In [10]:
#Lấy tất cả các danh mục cấp 2 - danh mục con
def crawl_all_categories_c2(category_name_c1, category_link_c1):


  url = category_link_c1

  response = requests.get(url, impersonate='chrome')

  soup = BeautifulSoup(response.content, 'html.parser')


  cate_books = soup.find_all('ol', id = 'children-categories')
  json_categories = []
  for cate in cate_books:
    title = cate.find_all('a')
    for i in title:
        category = {}
        category['Tên danh mục c1'] = category_name_c1
        category['Tên danh mục c2'] = i.get_text()
        category['Liên kết'] = i.attrs['href']
        category['Mã danh mục'] = i.attrs['cat_id']
        json_categories.append(category)

  df_categories_c2 = pd.json_normalize(json_categories)
  return df_categories_c2

---

## Craw dữ liệu

### Hàm crawl comments thông qua selenium

In [11]:
def crawl_comments_selenium(product_link, product_id):
    
    chrome_options = Options()
    chrome_options.add_argument("--disable-notifications")
    
    driver = webdriver.Chrome(options=chrome_options)
    
    driver.get(product_link)
    # cần refresh lại 1 lần trước khi scroll xuống do get lần đầu phần comment không laod
    driver.refresh()

    #scroll tới cuối trang để load phần comments
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # chờ 2-5s để load phần comment
    sleep(randint(2, 5))

    comments_of_product = []

    while True:
        try:
            #try catch để tránh trường hợp không có comment
            comments = driver.find_element(By.CLASS_NAME, 'comment_list').find_elements(By.TAG_NAME, 'li')
        except NoSuchElementException:
            driver.refresh()
            continue    
        for comment in comments:
            try:
                    content_comment = comment.text.split('\n')
                    dict_comment = {}
                    # không cần validate vì input comment đã được validate không được null
                    dict_comment['Mã sản phẩm'] = product_id

                    dict_comment['Tên khách hàng'] = content_comment[0]

                    dict_comment['Ngày'] = content_comment[1]

                    dict_comment['Nội dung đánh giá'] = content_comment[2]

                    comments_of_product.append(dict_comment)
            except:
                continue
        #try catch để tránh trường hợp không có nút next
        try:       
            next_button = driver.find_element(By.CSS_SELECTOR, 'a[onclick="prodComment.Page_change(\'next\')"]')
            next_button.click()
        except StaleElementReferenceException:
            break
        except NoSuchElementException:
            break
    driver.quit()        
    return comments_of_product

### Trích xuất thông tin sản phẩm

In [12]:

def extract_product(product_link, category_id):

    response = requests.get(product_link, impersonate='chrome')
    
    soup = BeautifulSoup(response.content, 'html.parser')

    product = {}

    comments_of_product = []
    product['Liên kết'] = product_link

    product_id_tag = soup.find('td', class_ ='data_sku')
    if product_id_tag:
        product_id = product_id_tag.get_text()
        product['Mã sản phẩm'] = product_id.strip()
    else:
        product['Mã sản phẩm'] = np.nan

    title_tag = soup.find('h1')
    product['Tên sản phẩm'] = title_tag.get_text().strip() if title_tag else np.nan

    product['Mã danh mục'] = category_id

    price_tag = soup.find('span', class_='price')
    product['Giá'] = price_tag.get_text().split('\xa0đ')[0].strip() if price_tag else np.nan

    old_price_tag = soup.find('p', class_='old-price')
    product['Giá Thị Trường'] = old_price_tag.find('span' , class_ = 'price').get_text().split('\xa0đ')[0].strip() if old_price_tag else np.nan

    sold_tag = soup.find('div', class_='product-view-qty-num')
    product['Số sản phẩm đã bán'] = sold_tag.get_text().split(' ')[2].strip() if sold_tag else np.nan

    nxb_tag = soup.find('td', class_='data_publisher')
    product['Nhà xuất bản'] = nxb_tag.get_text().strip() if nxb_tag else np.nan

    author_tag = soup.find('td', class_='data_author')
    product['Tác giả'] = author_tag.get_text().strip() if author_tag else np.nan

    qty_page_tag = soup.find('td', class_='data_qty_of_page')
    product['Số trang'] = qty_page_tag.get_text().strip() if qty_page_tag else np.nan

    # lấy đánh giá trung bình và số lượt đánh giá
    elems_review = soup.find('div', class_='product-view-tab-content-rating-chart')
    avg_review_rate = elems_review.text.split()[0]
    count_review = elems_review.text.split()[1]
        # slicing phần tử trong item
    try:
        avg_review_rate = float(avg_review_rate[:-2]) 
    except:
        avg_review_rate = np.nan
    try:
        count_review = int(count_review[1:]) 
    except:
            count_review = np.nan
    product['Đánh Giá trung bình'] = avg_review_rate 
    product['Số lượt đánh giá'] = count_review
    
    if product['Số lượt đánh giá'] != np.nan and product['Số lượt đánh giá'] > 0 and product['Số lượt đánh giá'] < 12:    
        chrome_options = Options()
        chrome_options.add_argument("--disable-notifications")
        driver = webdriver.Chrome(options=chrome_options)

        driver.get(product_link)
        driver.refresh()

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        sleep(randint(2, 5))

        comments = driver.find_element(By.CSS_SELECTOR, '.comment_list').find_elements(By.TAG_NAME, 'li')
        for comment in comments:
                try:
                
                    content_comment = comment.text.split('\n')
                    
                    dict_comment = {}

                    dict_comment['Mã sản phẩm'] = product['Mã sản phẩm']

                    dict_comment['Tên khách hàng'] = content_comment[0]

                    dict_comment['Ngày'] = content_comment[1]

                    dict_comment['Nội dung đánh giá'] = content_comment[2]

                    comments_of_product.append(dict_comment)
                except :
                    continue
                
        driver.quit()
        return product, comments_of_product
    
    elif product['Số lượt đánh giá'] > 12:
        comments_of_product = crawl_comments_selenium(product_link, product['Mã sản phẩm'] )
        return product, comments_of_product
    
    return product, False

In [13]:
extract_product('https://www.fahasa.com/bup-sen-xanh-tai-ban-2020.html?fhs_campaign=CATEGORY', 14)

({'Liên kết': 'https://www.fahasa.com/bup-sen-xanh-tai-ban-2020.html?fhs_campaign=CATEGORY',
  'Mã sản phẩm': '8935244826487',
  'Tên sản phẩm': 'Búp Sen Xanh (Tái Bản 2020)',
  'Mã danh mục': 14,
  'Giá': '61.200',
  'Giá Thị Trường': '72.000',
  'Số sản phẩm đã bán': '10k+',
  'Nhà xuất bản': 'NXB Kim Đồng',
  'Tác giả': 'Sơn Tùng',
  'Số trang': '364',
  'Đánh Giá trung bình': 5.0,
  'Số lượt đánh giá': 5},
 [{'Mã sản phẩm': '8935244826487',
   'Tên khách hàng': 'PhanLee',
   'Ngày': '03/08/2024',
   'Nội dung đánh giá': 'Sách hay lắm mn ơi,đọc sẽ hiểu được thêm về tuổi thơ của Bác Hồ và những nguyên nhân gì tác động khiến Bác ra đi tìm đường cứu nước,tuyệt vời nhe????❤️tiểu thuyết của Sơn Tùng cuốn mà hay lắm'},
  {'Mã sản phẩm': '8935244826487',
   'Tên khách hàng': 'Ph******',
   'Ngày': '19/09/2023',
   'Nội dung đánh giá': 'sách hay lắm ạ. cả nhà nên mua nhé. Fahasa giao nhanh xỉu,djhg;kldiyghio/sgirgfydhfddjidddddddddddddddddddnjgkhfv hbgfihposdisdkghbg'},
  {'Mã sản phẩm': '8

### Lấy tất cả sản phẩm trong danh mục con - child

In [18]:
def crawl_all_products_in_category(category_parent_name ,category_child_name, category_link, category_id):

    json_products = []

    json_comments = []

    total_products = 0
    #crawl nhiều nhất 10 pages mỗi page 48 sản phẩm co từng danh mục
    # page = 1

    # while True:
    # Set lại total cho progress bar ứng với số page crawl
    with tqdm(total=1,
        dynamic_ncols=True,
        unit=" trang",  # Đơn vị cho mỗi bước
        colour="blue",
        desc="Page: ",
        position=1
        ) as pbar:
        try:
            #test chỉ crawl 2 trang

            # for page in range(1, 10):
            for page in range(1, 2):
                response = requests.get(f"{category_link}{query_crawl}{page}" , impersonate='chrome')

                soup = BeautifulSoup(response.content, 'html.parser')

                products = soup.find('ul', class_='products-grid').find_all('li')

                if len(products) == 0 :
                    break
                else:
                    for i in range(0,len(products)):
                        # nếu series book thì bỏ qua
                        product_link = products[i].find('a').attrs['href']
                        series_book = r'(seriesbook\-index\-series)'
                        match = re.search(series_book, str(product_link))
                        if not match: 
                            
                            dict_product, comments_of_product = extract_product(product_link, category_id)

                            json_products.append(dict_product)
                        else : 
                            continue
                        if comments_of_product != False:
                            
                            json_comments.append(comments_of_product)

                        total_products+=1
                pbar.update(1)
        except:
            # nhảy vào khi hết trang trong range hoặc vòng while bị ngắt do hết page
            print(f'Đã crawl hết trang hiện có trên danh mục {category_parent_name} - {category_child_name}')   

        # page += 1

    json_comments = sum(json_comments, [])

    return json_products, json_comments

## Tiến hành chạy crawl

### 2 file csv chứa danh mục parent và child

In [16]:
#Tiến hành gọi hàm crawl_all_categories_c1 và crawl_all_categories_c2 để lấy dữ liệu
parent_categories = crawl_all_categories_c1()

#drop index 5 vì không phải danh mục sách
parent_categories = parent_categories.drop(index = 5).reset_index(drop = True)

child_categories = []

# Lấy các danh mục con của từng danh mục cha
for parent_category , links in zip(parent_categories['Tên danh mục c1'], parent_categories['Liên kết']): 
    child_category = crawl_all_categories_c2(parent_category, links)
    child_categories.append(child_category)

# concat các frame chứa trong mảng thành 1 mảng hoàn chỉnh, tự động reset index bằng ignore_index
child_categories = pd.concat(child_categories , ignore_index=True) 

# Xuất csv cho 2 frame chứa danh mục cấp parent và child
parent_categories.to_csv('./data/raw/fahasha_parent_categories.csv', encoding='utf-8-sig', index=False)
child_categories.to_csv('./data/raw/fahasha_child_categories.csv', encoding='utf-8-sig', index=False)


In [19]:
directory_paths = []
# bỏ comment để chạy tất cả các danh mục con
# crawl_categories = child_categories

# ta sẽ chỉ crawl 2 danh mục sách trong dự án này tương đương 55 danh mục sp
crawl_categories = child_categories[0:1]
with tqdm(total=int(crawl_categories.shape[0]),
        dynamic_ncols=True,
        unit=" bước",  
        colour="green",
        desc="Crawling: ",
        position=0
        ) as pbar:
    for index in  range(0,int(crawl_categories.shape[0])) :        
        #set tên danh mục c1 và c2 vào tiêu đề của progress bar
        pbar.set_postfix({"Danh Mục": child_categories['Tên danh mục c1'].loc[index] + '-' + child_categories['Tên danh mục c2'].loc[index]})

        category_c1_name = child_categories['Tên danh mục c1'].loc[index]
        category_c2_name = child_categories['Tên danh mục c2'].loc[index]

        category_link = child_categories['Liên kết'].loc[index]
        category_id = child_categories['Mã danh mục'].loc[index]

        json_products , json_comments = crawl_all_products_in_category(category_c1_name, category_c2_name, category_link, category_id)

        df_products = pd.json_normalize(json_products)
        df_comments = pd.json_normalize(json_comments)

        directory_c1 = unidecode(category_c1_name.lower()).replace(' ','_')
        directory_c2 = unidecode(category_c2_name.lower()).replace(' ','_')

        directory_path = Path(f'{local_path}/{directory_c1}/{directory_c2}')

        directory_path.mkdir(parents=True, exist_ok=True)
        directory_paths.append(directory_path)

        df_comments.to_csv(directory_path / 'comments.csv', encoding='utf-8-sig', index=False)
        df_products.to_csv(directory_path / 'products.csv', encoding='utf-8-sig', index=False)
        
        pbar.write(f"Đã lưu dữ liệu của danh mục {category_c1_name} - {category_c2_name} vào thư mục {directory_path}")
        pbar.update(1)
        

Crawling:   0%|          | 0/1 [00:00<?, ? bước/s]

Page:   0%|          | 0/1 [00:00<?, ? trang/s]

Đã lưu dữ liệu của danh mục Sách Trong Nước - Thiếu nhi vào thư mục data\raw\sach_trong_nuoc\thieu_nhi


### Gộp các file csv của các danh mục

#### Tạo mảng các path lưu trữ các file csv trong từng danh mục

In [ ]:
directory_paths = []
for index in  range(0,child_categories.shape[0]):
        # print('Crawling category:', child_categories['Tên danh mục c2'].loc[index ])
        category_c1_name = child_categories['Tên danh mục c1'].loc[index]
        category_c2_name = child_categories['Tên danh mục c2'].loc[index]

        directory_c1 = unidecode(category_c1_name.lower()).replace(' ','_')
        directory_c2 = unidecode(category_c2_name.lower()).replace(' ','_')

        directory_path = Path(f'{local_path}/{directory_c1}/{directory_c2}')

        directory_path.mkdir(parents=True, exist_ok=True)
        directory_paths.append(directory_path)


#### Hàm gộp csv

In [ ]:
def merged_csv(object):

    df_merged = pd.DataFrame()
    for path in directory_paths:
        #Do chưa crawl đầy đủ nên có thể có trường hợp không có file csv
        #Bỏ qua các trường hợp không có file csv
        try:
            df = pd.read_csv(path/f'{object}.csv')
            df_merged = pd.concat([df_merged, df], ignore_index=True)
        except:
            continue
    return df_merged

#### Tiến hành gộp

In [ ]:
products_merged = merged_csv('products')
products_merged.to_csv(f'{local_path}/fahasha_products.csv', encoding='utf-8-sig', index=False)

comments_merged = merged_csv('comments')
comments_merged.to_csv(f'{local_path}/fahasha_comments.csv', encoding='utf-8-sig', index=False)
